# Introduction

[SYOTools](https://github.com/tumlinson/luvoir_simtools) is a framework to enable users to create [Science Yield Optimization web tools](http://www.jt-astro.science/luvoir.html) for observatory design. It uses Bokeh & astropy to visualize exposure time calculators (ETCs), as well as other science cases, as a function of various observatory parameters. While SYOTools can be used independently of a particular observatory or science case, it has been primarily written to facilitate the design of [LUVOIR](https://asd.gsfc.nasa.gov/luvoir/).

SYOTools is divided into two main parts, contained in the `syotools.models` and `syotools.interface` subpackages. This Jupyter Notebook is intended as a walkthrough for using `syotools.models` to perform calculations. A future tutorial on using `syotools.interface` to design a web tool is in preparation; however, it will not be a similar Jupyter Notebook, as the interface framework is highly integrated with Bokeh Server. Instead, we will use Bokeh's `output_notebook` function, along with IPython interactors (as described [here](https://github.com/bokeh/bokeh/blob/0.12.13/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb)), to visualize the example calculations below. 

In [1]:
#Import Bokeh interface tools
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
output_notebook()

#Import numpy, syotools, and astropy.units
### NOTE: you will likely receive some pysynphot warnings when executing these imports, these are expected
import numpy as np
from syotools import cdbs #make sure that pysynphot can find its reference files!
from syotools.models import Telescope, Camera, Spectrograph #models for the observatory and instruments
from syotools.utils.jsonunit import str_jsunit #for printing JsonUnit and JsonSpectrum wrappers in a readable way
from syotools.utils import pre_encode, pre_decode
from syotools.spectra import SpectralLibrary
import astropy.units as u

Loading BokehJS ...

Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.


/Users/tumlinson/anaconda3/lib/python3.9/site-packages/pysynphot/units.py:824: RuntimeWarning: divide by zero encountered in log
  return -1.085736 * N.log(arg) + ABZERO


Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.


## Example 1: HDI ETC

From the LUVOIR design concept: *"The High Definition Imager (HDI) instrument is the primary astronomical imaging instrument for observations in the near UV through the near IR. The HDI design provides a 2 x 3 arcminute field-of-view, taking full advantage of the angular resolution provided by the telescope, and consists of two channels - an ultraviolet-visible (UVIS) channel covering 200 nm - 950 nm and a near-infrared (NIR) channel covering the range 800 nm - 2200 nm. The respective focal plane detector arrays provide Nyquist sampled images at 400 nm (2.73 mas/pixel) for UVIS imaging and at 1200 nm (8.20 mas/pixel) for NIR imaging."*

In this example, we will create a exposure time calculator for HDI, so that we can calculate the signal-to-noise ratio (SNR) for several possible template spectra. This approximates some of the functionality of [the official HDI ETC tool](http://jt-astro.science:5101/hdi_etc).

In [2]:
#Instatiate the observatory using syotools.models.Telescope
#This loads the default Telescope values, which are already based on the LUVOIR design
luvoir_ex1 = Telescope()
luvoir_ex1.aperture = pre_encode(15.08 * u.m) #set the telescope aperture for LUVOIR Architecture A

In [3]:
#Instantiate the instrument, using syotools.models.Camera, and link it with the telescope
#This loads the default Camera values, which are already based on the LUVOIR-HDI design
hdi = Camera()
luvoir_ex1.add_camera(hdi)

In [4]:
#Create a new photometric exposure for the camera
hdi_exposure = hdi.create_exposure()

#Print the available wave bands
pivotwave, bandpass = hdi.recover('pivotwave', 'derived_bandpass')

print("HDI wave bands:") 
for band, pwave, bpass in zip(hdi.bandnames, pivotwave, bandpass):
    print(u"   {:3s} - {:5.2f} ± {:5.2f}".format(band, pwave, bpass)) 

#Print the default template
default_hdi_template = SpectralLibrary[hdi_exposure.sed_id]
print("Current SED template: {}".format(default_hdi_template)) 

hdi_template_codes = ['fab', 'o5v', 'b5v', 'g2v', 'm2v', 'orion', 'elliptical', 'sbc', 'starburst', 'ngc1068']
available_hdi_templates = [SpectralLibrary[tc] for tc in hdi_template_codes]

HDI wave bands:
   FUV - 225.00 nm ± 45.00 nm
   NUV - 275.00 nm ± 55.00 nm
   U   - 336.00 nm ± 67.20 nm
   B   - 475.00 nm ± 95.00 nm
   V   - 606.00 nm ± 121.20 nm
   R   - 775.00 nm ± 155.00 nm
   I   - 850.00 nm ± 170.00 nm
   J   - 1260.00 nm ± 252.00 nm
   H   - 1600.00 nm ± 320.00 nm
   K   - 2220.00 nm ± 444.00 nm
Current SED template: Flat (AB)


In [5]:
#Create the Bokeh SED figure
hdi_sed = hdi_exposure.recover('sed')
hdi_sed.convert('abmag')
hdi_sed.convert('nm')
hdi_sed_fig = figure(height=300, width=600, title="SED", x_axis_label="Wavelength [nm]",
                 y_axis_label="AB Mag", y_range=(35, 21), x_range=(120, 2300))
hdi_sed_line = hdi_sed_fig.line(hdi_sed.wave, hdi_sed.flux, color='orange', line_width=3)

In [6]:
#Create the Bokeh SNR figure
hdi_snr = hdi_exposure.recover('snr')
hdi_snr_fig = figure(height=300, width=600, title="SNR", x_axis_label="Wavelength [nm]", 
                 y_axis_label="SNR", y_range=(0, 40), x_range=(120, 2300))
uv_hdi_snr = hdi_snr_fig.line(pivotwave[:2].value, hdi_snr[:2].value, color='orange', line_width=3)
vis_hdi_snr = hdi_snr_fig.line(pivotwave[2:-3].value, hdi_snr[2:-3].value, color='blue', line_width=3)
ir_hdi_snr = hdi_snr_fig.line(pivotwave[-3:].value, hdi_snr[-3:].value, color='red', line_width=3)

In [7]:
#Define the update callback function for interactive inputs
def hdi_update(template=default_hdi_template, aperture=12., exptime=1., v_magnitude=30.):
    #find the correct template code
    sed_id = hdi_template_codes[available_hdi_templates.index(template)]
    
    #turn off calculations until everything is updated
    hdi_exposure.disable()
    
    #update all of the telescope & exposure parameters
    hdi_exposure.exptime = exptime * u.h
    hdi_exposure.sed_id = sed_id
    luvoir_ex1.aperture = aperture * u.m
    hdi_exposure.renorm_sed(v_magnitude * u.ABmag)
    
    #turn calculations back on and recalculate based on updated parameters
    hdi_exposure.enable()
    
    #recover the recalculated values, and make sure everything is in the right units
    hdi_sed, hdi_snr = hdi_exposure.recover('sed', 'snr')
    hdi_sed.convert('nm')
    hdi_sed.convert('abmag')
    
    #sanitize the sed fluxes because some of the pysynphot spectra don't play nice
    hdi_flux = hdi_sed.flux
    hdi_flux[~np.isfinite(hdi_flux)] = v_magnitude
    
    #update the SED figure
    hdi_sed_fig.y_range.start = hdi_flux.max() + 5.
    hdi_sed_fig.y_range.end = hdi_flux.min() - 5.
    hdi_sed_line.data_source.data = {'x': hdi_sed.wave, 'y': hdi_flux}
    
    #update the SNR figure
    hdi_snr_fig.y_range.start = 0.
    hdi_snr_fig.y_range.end = max(1.3 * hdi_snr.value.max(), 5.)
    uv_hdi_snr.data_source.data['y'] = hdi_snr.value[:2]
    vis_hdi_snr.data_source.data['y'] = hdi_snr.value[2:-3]
    ir_hdi_snr.data_source.data['y'] = hdi_snr.value[-3:]
    
    #update the plots
    push_notebook()

In [8]:
#Show the plots
hdi_handle = show(column(hdi_sed_fig, hdi_snr_fig), notebook_handle=True)

In [9]:
#Create the interactive inputs
hdi_inputs = interact(hdi_update, template=available_hdi_templates, aperture=(2.0, 20.0), exptime=(0.1, 10.0, 0.1), 
         v_magnitude=(20.0, 35.0, 0.1))

interactive(children=(Dropdown(description='template', options=('Flat (AB)', 'O5V Star', 'B5V Star', 'G2V Star…